In [1]:
from datetime import datetime,timedelta
from collections import defaultdict,Counter
from pprint import pprint
from tqdm import tqdm
import re

import pymongo
from pymongo import InsertOne, DeleteMany, ReplaceOne, UpdateOne
from pymongo.errors import BulkWriteError

In [54]:
from fuzzywuzzy import fuzz

In [2]:
from nltk.corpus import stopwords

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import itertools

In [4]:
list_stopWords=list(set(stopwords.words('english')))

In [5]:
client = pymongo.MongoClient('localhost:27017')
db = client.tweet

In [6]:
#db.authenticate('admin','lixiepeng')

In [7]:
import pandas as pd
import spacy

In [8]:
nlp = spacy.load('en_core_web_md')

In [9]:
events = [e for e in db.current_event.find({'event.date':{'$gt':'2017-09-01','$lt':'2017-09-05'}},{'_id':1,'event.class':1,'event.date':1,'event.title':1,'event.description':1})]

In [10]:
events[0]

{'_id': ObjectId('59fbc9fb60b18848c5a55712'),
 'event': {'class': 'Armed attacks and conflicts',
  'date': '2017-09-02',
  'description': 'Seven people are killed and 13 are injured after suicide bombers hit a state-run power station near the northern city of Samarra, Iraq. The Islamic State of Iraq and the Levant claims responsibility for the attack. (Reuters)',
  'title': 'Iraqi Civil War'}}

In [11]:
events = [{'id':e['_id'],'class':e['event']['class'],'date':e['event']['date'],'title':e['event']['title'],'description':e['event']['description']} for e in events]

In [12]:
df_events = pd.DataFrame.from_records(events)

In [13]:
df_events.head()

,class,date,description,id,title
0,Armed attacks and conflicts,2017-09-02,Seven people are killed and 13 are injured aft...,59fbc9fb60b18848c5a55712,Iraqi Civil War
1,Business and economy,2017-09-02,Both the FBI and NSD declare that they possess...,59fbc9fb60b18848c5a55713,Trump Tower wiretapping allegations
2,Disasters and accidents,2017-09-02,Seven schoolgirls are killed and ten hospitali...,59fbc9fb60b18848c5a55714,2017 disasters in Kenya
3,International relations,2017-09-02,Australia and East Timor settle a dispute betw...,59fbc9fb60b18848c5a55715,Timor Gap
4,Law and crime,2017-09-02,Cambodian opposition leader Kem Sokha is arres...,59fbc9fb60b18848c5a55716,Law enforcement in Cambodia


In [18]:
df_events.iloc[0]['id']

ObjectId('59fbc9fb60b18848c5a55712')

In [19]:
tweets =  [tweet for tweet in db.paper.find({'event_id':df_events.iloc[0]['id']},{'tweet.standard_text':1})]

In [20]:
tweets

[{'_id': '903363851781099522',
  'tweet': {'standard_text': "IS leader al-Baghdadi 'probably still alive' claims US army commander in Iraq"}},
 {'_id': '903368426722381826',
  'tweet': {'standard_text': "IS leader alBaghdadi 'probably still alive' claims US army commander in Iraq via reddit"}},
 {'_id': '903396974715559936',
  'tweet': {'standard_text': 'ISIS claims suicide bombing that killed 29 at soccer match in Iraq'}},
 {'_id': '903400060377030656',
  'tweet': {'standard_text': "realDonaldTrump Why won't you talk to Amb. Bolton about Iraq he claims no one will let him near you? He says he gets stonewalled by staff."}},
 {'_id': '903438029041410048',
  'tweet': {'standard_text': "News IS leader al-Baghdadi 'probably still alive' claims US army commander in Iraq"}},
 {'_id': '903454755699138560',
  'tweet': {'standard_text': 'Iraq claims victory over ISIS in Tal Afar, but more battles loom via skinnergj'}},
 {'_id': '903470373886353408',
  'tweet': {'standard_text': 'IS claims suici

In [95]:
def reference_similatity(reference):
    doc_reference = nlp(reference['description'])
    print(reference['description'])
    tweets = []
    filter_dict = {'event_id':reference['id'],'tweet.lang':'en'}
    #filter_dict = {'event_id':reference['id'],'tweet.lang':'en','tweet.media.card_url':None}
    #filter_dict = {'event_id':reference['id'],'tweet.lang':'en','tweet.media.card_url':{'$ne':None}}
    #filter_dict = {'event_id':reference['id'],'tweet.lang':'en','f':'&f=news'}
    query_dict = {'tweet.standard_text':1}
    records  = [i for i in db.pos.find(filter_dict,query_dict)]+[i for i in db.paper.find(filter_dict,query_dict)]
    print(len(records))
    for tweet in records:
        tweet_id = tweet['_id']
        tweet_text = tweet['tweet']['standard_text'] 
        doc_tweet = nlp(tweet_text)
        coverage_num,jacard_similarity = entity_coverage(doc_reference,doc_tweet)
        char_match,token_match = fuzzy_string_matching(reference['description'],tweet_text)
        tweets.append((tweet_id,doc_reference.similarity(doc_tweet),tweet_text))
        #tweets.append((tweet_id,doc_reference.similarity(doc_tweet),coverage_num,jacard_similarity,char_match,token_match,tweet_text))  
    #tweets = [(,doc.similarity(nlp(tweet['tweet']['standard_text'])),entity_coverage,tweet['tweet']['standard_text']) for tweet in db.pos.find({'event_id':record['id']},{'tweet.standard_text':1})] + \
    #[(tweet['_id'],doc.similarity(nlp(tweet['tweet']['standard_text'])),entity_coverage,tweet['tweet']['standard_text']) for tweet in db.paper.find({'event_id':record['id']},{'tweet.standard_text':1})]
    tweets = sorted(tweets,key=lambda x:x[1],reverse=True)
    for i in tweets:
        print(i)

In [96]:
reference_similatity(df_events.iloc[11])

Syrian state television reports the Syrian Army reaches a point 3 kilometers from Deir ez-Zor, a city besieged by the Islamic State of Iraq and the Levant since 2014. (Reuters)
Two Russian soldiers, escorting the military staff ceasefire monitoring convoy, are killed in Deir ez-Zor Province by ISIL shelling. (Reuters)
207
('904744262499049472', 0.92306943746711545, 'Syrian army , allies 3 km from Deir al-Zor enclave: state TV ,IS is holding 93,000 civilians for years')
('904832661579026434', 0.91734328901533679, 'Two Russia |n contract soldiers were killed in the Syria |n province of Deir ez-Zor as a result ISIS mortar attack')
('904756045486202880', 0.91117196504155551, 'Syrian government forces are closing in on the eastern city of Deir al-Zour, which has been besieged by ISIS.')
('904832034144702466', 0.90087469217912353, 'Ministry of Defence: two Russian soldiers have died in the province Deir el-Zor when firing...')
('904838519742283777', 0.89892350959985856, 'Syrian government fo

In [97]:
def get_ents_set(doc):
    ents = [ent.text for ent in doc.ents]
    return set(ents)

In [42]:
def entity_coverage(doc_reference,doc_tweet):
    ents_reference = get_ents_set(doc_reference)
    ents_tweet = get_ents_set(doc_tweet)
    coverage_num = len(ents_reference & ents_tweet)
    jacard_similarity = coverage_num/len(ents_reference | ents_tweet)
    return coverage_num,jacard_similarity

In [60]:
def fuzzy_string_matching(text_reference,text_tweet):
    return fuzz.ratio(text_reference,text_tweet),fuzz.token_set_ratio(text_reference,text_tweet)